In [3]:
from functions import *

%load_ext autoreload
%autoreload 2

In [4]:
DATA_PATH = '/home/helvecioneto/disco1/dados/goes16/mmh/'
DATA_NAME = 'goes16'
DATA_PATTERN = '%%YYYY%%mm%%dd_%%HH%%MM.nc'
TRACK_PATH = '/home/helvecioneto/disco1/tracks/goes16mmh/track/'
FORECAST_PATH = '/home/helvecioneto/disco1/tracks/goes16mmh/forecast/'
TIME_DELTA = '10min'

In [5]:
# PATHS
timestamp = get_latest_timestamp(TRACK_PATH)
pth_trck_bound, pth_trck_vec, pth_trck_trj = get_geotrack_paths(TRACK_PATH, timestamp)
pth_fcst_bound, pth_fcst_vec, pth_fcst_trj = get_geofrcst_paths(FORECAST_PATH, timestamp, TIME_DELTA, 1)
data_path = get_data_paths(timestamp, DATA_PATH+timestamp.strftime('%Y/%m/%d/'), DATA_PATTERN)

In [6]:
# READ
data = read_data(DATA_NAME)(data_path)
trck_bound = read_geoframe(pth_trck_bound)
trck_vec = read_geoframe(pth_trck_vec)
trck_trj = read_geoframe(pth_trck_trj)
fcst_bound = read_geoframe(pth_fcst_bound)
fcst_vec = read_geoframe(pth_fcst_vec)
fcst_trj = read_geoframe(pth_fcst_trj)


In [98]:
import plotly.graph_objects as go
import xarray as xr
import plotly.express as px
import datashader as ds
import datashader.transfer_functions as tf
from colorcet import rainbow
import numpy as np

In [107]:
# Read data_path using xarray
dset = xr.open_dataset(data_path)
# Convert xarray to dataset
df = dset.to_dataframe()
# Remove NaN values
df = df.dropna().reset_index()
# Convert dataset to datashader format
cvs = ds.Canvas(plot_width=910, plot_height=910)
agg = cvs.points(df, 'lon', 'lat')
coords_lat, coords_lon = agg.coords['lat'].values, agg.coords['lon'].values
coordinates = [[coords_lon[0], coords_lat[0]],
               [coords_lon[-1], coords_lat[0]],
               [coords_lon[-1], coords_lat[-1]],
               [coords_lon[0], coords_lat[-1]]]
img = tf.shade(agg, cmap=rainbow)[::-1].to_pil()

In [108]:
## Create a figure with plotly.graph_objects
fig = go.Figure()

fig = px.scatter_mapbox(df[:1], lat='lat', lon='lon', zoom=12)
fig.update_layout(
                 mapbox_layers = [
                {
                    "sourcetype": "image",
                    "source": img,
                    "coordinates": coordinates
                }]
)


fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=-50, mapbox_center_lat=-20, mapbox_zoom=3)
fig.update_layout(height=600,width=900, margin={"r":0,"t":40,"l":100,"b":0,"pad":0})
fig.show()